In [1]:
cd ..

c:\Users\14102\Brown\Realizing_Rights\RealizingRights


In [2]:
from utilities.web_utils import closest_link_match, make_https, prepend_root_to_url, make_driver_utils
from url_scraper.link_data import LinkData
from main import main
from objects.scrape_lists import blacklist_terms, link_keywords, board_meeting_keywords, social_media_sites
from url_scraper.web_scraper import DistrictWebsiteScraper
from utilities.data_utils import format_input_district_df, format_output_boe_df

import os
import re
from concurrent.futures.thread import ThreadPoolExecutor
import queue
import pandas as pd

In [ ]:
# source_info = {'path': 'data/USSchoolDistrictWebsiteInfo.xlsx', 'sheet': 'ELSI Export', 'head_row': 6}
# write_file_path = 'data/SampleOutput.csv'
    
# main(source_info=source_info, write_file_path=write_file_path, verbose=False, max_dist_runs=None, out_file_name=write_file_path)

In [ ]:
current_output = pd.read_csv('data/SampleOutput.csv')

def check_malformed(url: str) -> bool:
    if len([char.start() for char in re.finditer('//', url)]) > 1:
        return True
    elif 'javascript' in url:
        return True
    else:
        return False

# current_output['Board Meeting URL Link'] = current_output.apply(lambda x: str(x['Board Meeting URL Link']), axis=1)
current_pop_output = current_output[~current_output['Board Meeting URL Link'].isna()].reset_index(drop=True)
current_unpop_output = current_output[current_output['Board Meeting URL Link'].isna()].reset_index(drop=True)

current_pop_output['BadBoardURL'] = current_pop_output.apply(lambda x: check_malformed(url=x['Board Meeting URL Link']), axis=1)
malformed_brd_mt_df = current_pop_output[current_pop_output['BadBoardURL']]
current_pop_output = current_pop_output[~current_pop_output['BadBoardURL']]
current_pop_output.drop('BadBoardURL', axis=1, inplace=True)
current_pop_output = current_pop_output.append(current_unpop_output)
malformed_brd_mt_df

In [4]:
current_output = pd.read_csv('data/SampleOutput.csv')
missing_brd_mt_df = current_output[current_output['Board Meeting URL Link'].isna()].reset_index(drop=True)

In [ ]:
pipeline = queue.Queue()
full_df = pd.DataFrame()
urls_processed = 0

def reprocess_missing_data(source: pd.DataFrame, write_file_path: str, verbose: bool, max_dist_runs: int) -> None:
    global full_df

    district_df_to_use = source[:max_dist_runs]
    # Scrape data
    
    url_list_to_process = list(district_df_to_use['URL'])
    num_urls = len(url_list_to_process)
    url_id_zip = [(url, a_id) for url, a_id in zip(list(district_df_to_use['URL']), list(district_df_to_use['District ID']))]

    with ThreadPoolExecutor(max_workers=12) as executor:

        _ = executor.submit(build_output_df, num_urls)
        executor.map(scrape_district, url_id_zip)

    full_df.to_csv(write_file_path, index=False)


def scrape_district(url_id_tuple: tuple):
    global urls_processed
    print(f'Parsing {url_id_tuple[0]}...')
    try:
        district_scraper = DistrictWebsiteScraper(url=url_id_tuple[0], agency_id=url_id_tuple[1], verbose=False)
        district_scraper.find_board_meeting_and_social_media_links()
        pipeline.put(district_scraper.url_data)
        print(f'{url_id_tuple[0]} Done')
    except Exception as e:
        print(f"{url_id_tuple[0]} Error: {e}")
        urls_processed += 1


def build_output_df(num_urls: int):
    global full_df
    global urls_processed
    while urls_processed < num_urls:
        url_info_dict = pipeline.get()
        url_info_row = format_output_boe_df(url_data=url_info_dict, social_media_sites=social_media_sites)
        full_df = full_df.append(url_info_row)
        urls_processed += 1
        print(urls_processed)


In [ ]:
write_file_path = 'data/reprocessed.csv'

reprocess_missing_data(source=malformed_brd_mt_df, write_file_path=write_file_path, max_dist_runs=9999, verbose=False)


In [5]:
source_info = {'path': 'data/USSchoolDistrictWebsiteInfo.xlsx', 'sheet': 'ELSI Export', 'head_row': 6}
xlsx_input = pd.read_excel(io=source_info['path'], sheet_name=source_info['sheet'], header=source_info['head_row'])
district_df = format_input_district_df(dist_df=xlsx_input)
pop_output = current_output[~current_output['Board Meeting URL Link'].isna()]


In [ ]:
full_df_info = district_df.merge(full_df, left_on='Agency ID', right_on='District ID')

In [ ]:
# updated_output_df = pop_output.append(full_df_info)
updated_output_df = current_pop_output.append(full_df_info)
updated_output_df

In [ ]:
# updated_output_df.to_csv('data/SampleOutput.csv', index=False)

In [ ]:
current_output = pd.read_csv('data/SampleOutput.csv')
